In [ ]:
import pandas as pd
import networkx as nx
from pathlib import Path

prefix = 'data/umimematikucz-system_'

kc = pd.read_csv(Path(prefix + "kc.csv", ), sep=';', header=0, index_col=0)
follow = pd.read_csv(Path(prefix + "kc_follow.csv", ), sep=';', header=0, index_col=0)
ps = pd.read_csv(Path(prefix + "ps.csv", ), sep=';', header=0, index_col=0)

G = nx.Graph()

G.add_nodes_from([(idx, {'name':name}) for idx, name in zip(kc.index, kc['name'])])
G.add_node(0, name='root')
G.add_edges_from([(idx, parent) for idx, parent in zip(kc.index, kc['parent'])])
G.add_edges_from([(first, second) for first, second in follow[['kc1', 'kc2']].to_numpy()])
pos = nx.spectral_layout(G, scale=400, center=(200, 200))

In [ ]:

elements = []

elements += [
    {'data': {'id': str(i), 'label': params['name']}, 'position': {'x': int(pos[i][0]), 'y': int(pos[i][1])}} for i, params in G.nodes(data=True)
]

elements += [
    {'data': {'source': str(s), 'target': str(t)}} for s, t in G.edges()
]


In [ ]:
import json
import dash_cytoscape as cyto
from dash import html, Input, Output
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)

styles = json.load(open('styles.json', 'r'))

cyto.load_extra_layouts()

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape',
        elements=elements,
        style=styles['cytoscape'],
        layout={'name': 'dagre'},
        responsive=True,
    )
])

if __name__ == "__main__":
    app.run_server(debug=True)